# Training a classification model with ResNet  
In this tutorial we will use the Resnet Model Adapter to inference and train on custom data.  
If you don't have the following packages, you'll need to install. The Torch Model Adapter will use them later:  
torch  
torchvision  
imgaug  
scikit-image<0.18  
  


In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import dtlpy as dl
from dtlpy.ml import train_utils


e:\shabtay\fonda\venv\lib\site-packages\dtlpy\services\api_client.py:1193: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead
  raise_on_status=False


## Get Global Model and Pretrained Snapshot  
We start by getting tne model and an ImageNet pretrained model  


In [2]:
model = dl.models.get(model_name='ResNet')
snapshot = model.snapshots.get('pretrained-resnet18')
model.snapshots.list().to_df()


e:\shabtay\fonda\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,creator,name,description,is_global,status,tags,configuration,version,context,modelId,datasetId,createdAt,bucket,ontologySpec
0,616d70ed451677b35347ee34,yair@dataloop.ai,pretrained-resnet18,resnset18 pretrained on imagenet,None,created,"[pretrained, imagenet]","{'weights_filename': 'model.pth', 'classes_fil...",1.0.0,{'org': 'd25cecad-34f7-4e38-9750-bd9a1d387bfd'...,616d70c57136ca40bca78ad6,None,2021-10-18T13:04:45.480Z,"{'type': 'gcs', 'gcsProjectName': 'viewo-main'...","{'labels': ['tench', 'goldfish', 'great white ..."
1,6280bde54b152c24a698ee26,or@dataloop.ai,sheep-soft-augmentations,resnset18 pretrained on imagenet,None,created,"[pretrained, imagenet]","{'weights_filename': 'model.pth', 'classes_fil...",1.0.1,{'org': 'd25cecad-34f7-4e38-9750-bd9a1d387bfd'...,616d70c57136ca40bca78ad6,6280bda5ea2a6ca88830589b,2021-10-18T13:04:45.480Z,"{'type': 'item', 'itemId': '6280bde4c31b850518...","{'labels': ['tench', 'goldfish', 'great white ..."
2,616d70f37136cad9e8a78ad7,yair@dataloop.ai,pretrained-resnet50,resnset50 pretrained on imagenet,None,created,"[pretrained, imagenet]","{'weights_filename': 'model.pth', 'classes_fil...",1.0.0,{'org': 'd25cecad-34f7-4e38-9750-bd9a1d387bfd'...,616d70c57136ca40bca78ad6,None,2021-10-18T13:04:51.467Z,"{'type': 'gcs', 'gcsProjectName': 'viewo-main'...","{'labels': ['tench', 'goldfish', 'great white ..."
3,616ef7be950f5dccfc9e57b8,pipelines-reg@dataloop.ai,resnet18-label-sug-pretrained-recipe-60350315b...,resnset18 pretrained on imagenet,None,created,"[pretrained, imagenet]","{'weights_filename': 'model.pth', 'classes_fil...",1.0.1,{'org': 'd25cecad-34f7-4e38-9750-bd9a1d387bfd'...,616d70c57136ca40bca78ad6,None,2021-10-19T16:52:14.537Z,"{'type': 'item', 'itemId': '616ef7be930c82bada...","{'labels': ['tench', 'goldfish', 'great white ..."
4,616ef943950f5d2c659e57ba,pipelines-reg@dataloop.ai,resnet18-label-sug-pretrained-recipe-616be3db4...,resnset18 pretrained on imagenet,None,created,"[pretrained, imagenet]","{'weights_filename': 'model.pth', 'classes_fil...",1.0.1,{'org': 'd25cecad-34f7-4e38-9750-bd9a1d387bfd'...,616d70c57136ca40bca78ad6,None,2021-10-19T16:58:43.796Z,"{'type': 'item', 'itemId': '616ef943930c82bada...","{'labels': ['tench', 'goldfish', 'great white ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,621e24afa7c041a172862d7c,pipelines@dataloop.ai,resnet18-label-sug-pretrained-recipe-621e2319c...,My snapshot,None,created,[label-suggest],"{'batch_size': 16, 'start_epoch': 0, 'num_epoc...",1.0.0,{'org': 'd25cecad-34f7-4e38-9750-bd9a1d387bfd'...,616d70c57136ca40bca78ad6,None,2022-03-01T13:50:39.909Z,"{'type': 'item', 'itemId': '621e249b4aa3db5d7e...","{'labels': [], 'ontologyId': 'null'}"
114,621e3948a7c04117cd862d84,pipelines@dataloop.ai,resnet18-label-sug-pretrained-recipe-621e391d8...,My snapshot,None,created,[label-suggest],"{'batch_size': 16, 'start_epoch': 0, 'num_epoc...",1.0.0,{'org': 'd25cecad-34f7-4e38-9750-bd9a1d387bfd'...,616d70c57136ca40bca78ad6,None,2022-03-01T15:18:32.709Z,"{'type': 'item', 'itemId': '621e39391bdd033c7f...","{'labels': ['Test', 'Test2'], 'ontologyId': 'n..."
115,624ecd80c7e35d6ff5e24d21,pipelines@dataloop.ai,resnet18-label-sug-pretrained-recipe-61dc00b54...,My snapshot,None,created,[label-suggest],"{'batch_size': 16, 'start_epoch': 0, 'num_epoc...",1.0.0,{'org': '7db04660-d106-4ec2-879e-ba48fb0ca2b0'...,616d70c57136ca40bca78ad6,None,2022-04-07T11:39:44.694Z,"{'type': 'item', 'itemId': '624ecd70c07eba9a96...","{'labels': ['dog', 'eye', 'as', 's', 'aaaaaaa'..."
116,625d8032ff09e1357a3ad057,bot.bb7c3624-5ce7-48e0-a317-22662cfae46d@bot.d...,resnet18-label-sug-pretrained-recipe-625d7d47e...,My snapshot,None,created,[label-suggest],"{'batch_size': 16, 'start_epoch': 0, 'num_epoc...",1.0.0,{'org': 'd25cecad-34f7-4e38-9750-bd9a1d387bfd'...,616d70c57136ca40bca78ad6,None,2022-04-18T15:13:54.015Z,"{'type': 'item', 'itemId': '625d801a52e3262060...","{'labels': ['Test1', 'Test2', 'Test3'], 'ontol..."


### Run Pretrained Model  
We will "build" to model adapter to get the model code locally and the create an instance of the ModelAdapter class.  
After that, we load the pretrained snapshot into the model adapter.  


In [3]:
adapter = model.build()
adapter.load_from_snapshot(snapshot=snapshot)


e:\shabtay\fonda\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Get an item and predict with upload.  
You can also open the item in the platform to view and edit annotations easily.  


In [0]:
item = dl.items.get(item_id='611e174e4c09acc3c5bb81d3')
annotations = adapter.predict_items([item], with_upload=True)
image = np.asarray(Image.open(item.download()))
plt.imshow(item.annotations.show(image,
                                 thickness=5))
print('Classification: {}'.format(annotations[0][0].label))


## Train on new dataset  
We will use a public sheep face dataset. We create a project and a dataset and upload the data with 4 labels of sheep.  
NOTE: You might need to change the location of the items (should point to the root of the documentation repository)  


In [4]:
project = dl.projects.get('Sheep Face - Model Mgmt')
dataset = project.datasets.get('Sheep Face')
# dataset.to_df()
# _ = dataset.items.upload(local_path='../../../../assets/sample_datasets/SheepFace/items/*',
#                          local_annotations_path='../../../../assets/sample_datasets/SheepFace/json')
# dataset.add_labels(label_list=['Marino', 'Poll Dorset', 'Suffolk', 'White Suffolk'])


e:\shabtay\fonda\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Now we'll run the "prepare_dataset" method. This will clone and freeze the dataset (so that we'll be able to reproduce the training and keep a snapshot of the data).  
The cloned dataset will be split into subsets (using DQL or percentage). In this examples, we'll use a 80/20 train validation split.  
After that we clone the pretrained snapshot to have a starting point for the fine-tuning.  
The snapshot's configuration will determine some runtime configurations, for instance, we will train for only 2 epochs.  


In [17]:
partitions = {dl.SnapshotPartitionType.TRAIN: 0.8,
              dl.SnapshotPartitionType.VALIDATION: 0.2}
cloned_dataset = train_utils.prepare_dataset(dataset,
                                             filters=None,
                                             partitions=partitions)
snapshot_name = 'sheep-soft-augmentations'
# create an Item Bucket to save snapshot in your project
# bucket = project.buckets.create(bucket_type=dl.BucketType.ITEM,
#                                 model_name=model.name,
#                                 snapshot_name=snapshot_name)
# new_snapshot = snapshot.clone(snapshot_name=snapshot_name,
#                               dataset_id=cloned_dataset.id,
#                               project_id=project.id,
#                               bucket=bucket,
#                               configuration={'batch_size': 16,
#                                              'start_epoch': 0,
#                                              'num_epochs': 2,
#                                              'input_size': 256,
#                                              'id_to_label_map': {(v - 1): k for k, v in
#                                                                  dataset.instance_map.items()}
#                                              },
                              
#                              )
new_snapshot = project.snapshots.get(snapshot_name=snapshot_name)


[2022-05-15 13:15:30][WAR][dtlpy:v1.57.10][ml.train_utils:37] Cloned dataset already exist. Using it...


We'll load the new un-trained snapshot to the adapter and prepare the training local dataset  


In [18]:
adapter.load_from_snapshot(snapshot=new_snapshot)
root_path, data_path, output_path = adapter.prepare_training()


[2022-05-15 13:16:03][WAR][dtlpy:v1.57.10][ml.base_model_adapter:66] Replacing snapshot from 'pretrained-resnet18' to 'sheep-soft-augmentations'
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:38<00:00,  2.61it/s]
[2022-05-15 13:16:45][WAR][dtlpy:v1.57.10][entities.item:108] Item has been fetched from a dataset that is not belong to it
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.40it/s]
[2022-05-15 13:17:24][WAR][dtlpy:v1.57.10][entities.item:108] Item has been fetched from a dataset that is not belong to it
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

## Start The Train  
Now We have the model, the snapshot, and the data ready. We are ready to train.  


In [19]:
print("Training {!r} with snapshot {!r} on data {!r}".format(model.name, new_snapshot.id, data_path))
adapter.train(data_path=data_path,
              output_path=output_path)


[2022-05-15 13:17:36][WAR][dtlpy:v1.57.10][repositories.recipes:159] Deprecation Warning - return type will be pageEntity from version 1.46.0 not a list


Training 'ResNet' with snapshot '6280d25491ba780d27ac6dc3' on data 'C:\\Users\\Shabtay\\.dataloop\\training\\6280d25491ba780d27ac6dc3_sheep-soft-augmentations\\2022-05-15-131605\\datasets\\6280c489e72e76a6d35e4c51'


[2022-05-15 13:17:36][WAR][dtlpy:v1.57.10][repositories.ontologies:230] Deprecation Warning - return type will be pageEntity from version 1.46.0 not a list
  0%|                                                                                                                                                                                                           | 0/84 [00:00<?, ?batch/s]e:\shabtay\fonda\venv\lib\site-packages\imgaug\imgaug.py:145: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  return isinstance(val, collections.Iterable)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 22.33batch/s, accuracy=86.6, loss=0.351]


## Save the Snapshot  
We will save the locally-trained snapshot and upload the trained weights to the Item Bucket.  
This will ensure we have everything in the Dataloop platform and everyone can use our trained snapshot.  


In [20]:
adapter.save_to_snapshot(local_path=output_path,
                         replace=True)


0it [00:00, ?it/s]
  0%|                                                                                                                                                                                                         | 0.00/42.7M [00:00<?, ?B/s]
  0%|▎                                                                                                                                                                                                | 64.0k/42.7M [00:00<07:21, 101kB/s]
  0%|▊                                                                                                                                                                                                 | 192k/42.7M [00:00<02:22, 312kB/s]
  1%|█▍                                                                                                                                                                                                | 320k/42.7M [00:00<01:47, 414kB/s]
  1%|█▉                                  

We can also list our bucket's content, and add more files that are needed for loading/running the snapshot  


In [22]:
adapter.snapshot.bucket.list_content().to_df()


,annotated,filename,name,type,id,hidden,dir,annotationsCount,dataset,createdAt,datasetId,creator
0,False,/artifacts/models/ResNet/snapshots/sheep-soft-...,model.pth,file,6280d3667c2e42d85a538c94,False,/artifacts/models/ResNet/snapshots/sheep-soft-...,0,https://rc-gate.dataloop.ai/api/v1/datasets/62...,2022-05-15T10:18:14.000Z,6280c4509a7b5c46d4632ee9,or@dataloop.ai


## Predict On Our New Trained Snapshot  
We will load our snapshot and visualize some items' predictions  


In [23]:
item = dl.items.get(item_id='611e174e4c09acc3c5bb81d3')
annotations = adapter.predict_items([item], with_upload=True)
image = Image.open(item.download())
plt.imshow(item.annotations.show(np.asarray(image),
                                 thickness=5))
print('Classification: {}'.format(annotations[0][0].label))


e:\shabtay\fonda\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[2022-05-15 13:18:33][ERR][dtlpy:v1.57.10][services.api_client:1327] [Response <404>][Reason: Not Found][Text: {"statusCode":404,"message":"item not found"}]


NotFound: ('404', 'item not found')